# Greedy 

Os resultados do processamento (pasta [out](https://github.com/thyarles/unb-fmc-nlp/tree/main/aula_3/atividade_3/out)):

* Base `classic4.csv`
    * Greedy Search: [NB](https://github.com/thyarles/unb-fmc-nlp/tree/main/aula_3/atividade_3/out/classic4_greedy_nb.csv) | [LR](https://github.com/thyarles/unb-fmc-nlp/tree/main/aula_3/atividade_3/out/classic4_greedy_lr.csv) | [SVM](https://github.com/thyarles/unb-fmc-nlp/tree/main/aula_3/atividade_3/out/classic4_greedy_svm.csv)
    * [Métricas]((https://github.com/thyarles/unb-fmc-nlp/tree/main/aula_3/atividade_3/out/classic4_greedy_metrics.csv))

* Base `Dmoz-Sports.csv`
    * Greedy Search: [NB](https://github.com/thyarles/unb-fmc-nlp/tree/main/aula_3/atividade_3/out/Dmoz-Sports_greedy_nb.csv) | [LR](https://github.com/thyarles/unb-fmc-nlp/tree/main/aula_3/atividade_3/out/Dmoz-Sports_greedy_lr.csv) | [SVM](https://github.com/thyarles/unb-fmc-nlp/tree/main/aula_3/atividade_3/out/Dmoz-Sports_greedy_svm.csv)
    * [Métricas]((https://github.com/thyarles/unb-fmc-nlp/tree/main/aula_3/atividade_3/out/Dmoz-Sports_greedy_metrics.csv))

## Definição dos hiper parâmetros

In [1]:
# Importa Pandas e find_best_hyperparameter
import pandas as pd
from find_best_hyperparameters import AtividadeTres

# Carrega Atividade 3
a3 = AtividadeTres(26)

# Define hiper parâmetros para Greedy Search
hyper_params = {}

hyper_params['nb'] = {
    'model_name': 'MultinomialNB',
    'params': a3.create_param_grid({   
        # Explicar alpha
        'alpha': [0.1, 0.25, 0.5, 0.75, 1.0, 2.0, 5.0],
        # Aprender probabilidades anteriores de classe
        'fit_prior': [True, False]
    })
}

hyper_params['lr'] = {
    'model_name': 'LogisticRegression',
    'params': a3.create_param_grid({
        # Força da regularização
        'C': [0.1, 0.25, 1.0, 5.0, 10.0, 20.0],
        # Algoritmos de otimização
        'solver': ['lbfgs', 'liblinear', 'sag'],
        # Tipos de regularização
        'penalty': ['l1', 'l2', 'elasticnet', None],
        # Iterações máximas
        'max_iter': [100, 200, 500, 1000]
    })
}

hyper_params['svm'] = {
    'model_name': 'LinearSVC',
    'params': a3.create_param_grid({
        # Força da regularização
        'C': [0.01, 0.1, 1, 5, 10, 50, 100],
        # Função de perda
        'loss': ['hinge', 'squared_hinge'],
        # Iterações máximas
        'max_iter': [100, 250, 500, 1000, 5000],
        # Critério de tolerância para parar
        'tol': [1e-3, 1e-4, 1e-5]
    })
}

Classe iniciada com semente 26!
Hiper parâmetros montados: dict_keys(['alpha', 'fit_prior'])
Hiper parâmetros montados: dict_keys(['C', 'solver', 'penalty', 'max_iter'])
Hiper parâmetros montados: dict_keys(['C', 'loss', 'max_iter', 'tol'])


## Base Classic4.csv

In [2]:
# Leitura da base Classic4.csv
BASE='classic4'
FILENAME='classic4.csv'
cstr = a3.load_data(FILENAME)
cstr.head()

,file_name,text,class
0,cacm.001232,Representation of the Standard ECMA 7-Bit Code...,cacm
1,cacm.002901,An Encoding Method for Multifield Sorting and ...,cacm
2,cacm.003171,Line Numbers Made Cheap A technique is describ...,cacm
3,cacm.001938,Some Criteria for Time-Sharing System Performa...,cacm
4,cacm.000233,Abstracts-Additional Nuclear Reactor Codes,cacm


In [3]:
# Separação dos dados de treino e teste
train, test, series = a3.split_data(data=cstr, text_col='text', class_col='class', test_size=0.2)

# Salva resultados
a3.save_data_frame(train, BASE + '_train.csv')
a3.save_data_frame(test, BASE + '_test.csv')

# Mostra primeiras linhas do teste
test.head()

Dados divididos com tamanho do teste 0.20 e semente 26!
Dicionário de séries: dict_keys(['X_train', 'X_test', 'y_train', 'y_test'])
Arquivo out/classic4_train.csv salvo com sucesso!
Arquivo out/classic4_test.csv salvo com sucesso!


,text,class
1442,The Role of Programming in a Ph.D. Computer Sc...,cacm
2527,Anomalous Behavior of the Fifty-Percent Rule i...,cacm
6391,transient familial neonatal hyperbilirubinemia...,med
174,AUTOMAST: Automatic Mathematical Analysis and ...,cacm
5875,theory of stagnation point heat transfer in di...,cran


In [4]:
# Greedy Search para NB, LR and SVM
best_greedy = {}
greedy = None

for item in hyper_params:
    # Testa parâmetros com Greedy Search
    greedy = pd.DataFrame(a3.greedy_search(
        model_name=hyper_params[item]['model_name'],
        param_grid=hyper_params[item]['params'],
        series=series
    ))
    # Salva resultados em arquivo
    a3.save_data_frame(greedy, BASE + '_greedy_' + item + '.csv')
    # Resgata melhor resultado para treinamento
    max_score = greedy['score'].max()
    max_rows = greedy[greedy['score'] == max_score]
    best_greedy[item] = max_rows.sample(1).reset_index(drop=True).iloc[0].to_dict()
    
# Exibe valores no notebook
pd.DataFrame(best_greedy)

Arquivo out/classic4_greedy_nb.csv salvo com sucesso!
Arquivo out/classic4_greedy_lr.csv salvo com sucesso!
Arquivo out/classic4_greedy_svm.csv salvo com sucesso!


,nb,lr,svm
alpha,0.1,NaN,NaN
fit_prior,True,NaN,NaN
score,0.98203,0.997886,0.995067
C,NaN,0.1,0.01
solver,NaN,sag,NaN
penalty,NaN,l2,NaN
max_iter,NaN,500,250
loss,NaN,NaN,squared_hinge
tol,NaN,NaN,0.001


In [5]:
# Treina modelo com melhores parâmetros
metrics = {}

for item in best_greedy:
    # Remove key score
    try:
        best_greedy[item].pop('score')
    except:
        pass
    metrics[item] = a3.train(model_name=item, params=best_greedy[item], series=series, base=BASE)

a3.save_data_frame(pd.DataFrame(metrics), BASE + '_metrics.csv')

# Exibe valores no notebook
pd.DataFrame(metrics)

Arquivo out/classic4_metrics.csv salvo com sucesso!


,nb,lr,svm
f1_score_macro,0.963803,0.961813,0.958692
f1_score_micro,0.958421,0.958421,0.954193
accuracy,0.958421,0.958421,0.954193


## Base Dmoz-Sports.csv

In [6]:
# Leitura da base Dmoz-Sports.csv
BASE='Dmoz-Sports'
FILENAME='Dmoz-Sports.csv'
cstr = a3.load_data(FILENAME)
cstr.head()

,file_name,text,class
0,3191005.txt,"Bryant Bulldogs Official site with news, sched...",Baseball
1,3193010.txt,Hirschbeck v. Alomar Covers a legal case betwe...,Baseball
2,3190744.txt,Mountaineers Baseball Official site. Includes ...,Baseball
3,3193695.txt,"M.S.H. Little League Contains news items, a li...",Baseball
4,3193486.txt,Quotations: Bob Uecker Baseball Almanac collec...,Baseball


In [7]:
# Separação dos dados de treino e teste
train, test, series = a3.split_data(data=cstr, text_col='text', class_col='class', test_size=0.2)

# Salva resultados
a3.save_data_frame(train, BASE + '_train.csv')
a3.save_data_frame(test, BASE + '_test.csv')

# Mostra primeiras linhas do teste
test.head()

Dados divididos com tamanho do teste 0.20 e semente 26!
Dicionário de séries: dict_keys(['X_train', 'X_test', 'y_train', 'y_test'])
Arquivo out/Dmoz-Sports_train.csv salvo com sucesso!
Arquivo out/Dmoz-Sports_test.csv salvo com sucesso!


,text,class
5712,"Milton Keynes Kings Fansite includes fixtures,...",Hockey
1130,Colorado State Bowling Proprietors Association...,Bowling
2236,Bluegrass Cycling Club Based in Lexington. Inc...,Cycling
6995,Seong's Martial arts Kendo Academy Sung Moo Kw...,Martial
1020,Andy B's Tulsa Bowling and entertainment cente...,Bowling


In [8]:
# Greedy Search para NB, LR and SVM
best_greedy = {}
greedy = None

for item in hyper_params:
    # Testa parâmetros com Greedy Search
    greedy = pd.DataFrame(a3.greedy_search(
        model_name=hyper_params[item]['model_name'],
        param_grid=hyper_params[item]['params'],
        series=series
    ))
    # Salva resultados em arquivo
    a3.save_data_frame(greedy, BASE + '_greedy_' + item + '.csv')
    # Resgata melhor resultado para treinamento
    max_score = greedy['score'].max()
    max_rows = greedy[greedy['score'] == max_score]
    best_greedy[item] = max_rows.sample(1).reset_index(drop=True).iloc[0].to_dict()
    
# Exibe valores no notebook
pd.DataFrame(best_greedy)

Arquivo out/Dmoz-Sports_greedy_nb.csv salvo com sucesso!
Arquivo out/Dmoz-Sports_greedy_lr.csv salvo com sucesso!
Arquivo out/Dmoz-Sports_greedy_svm.csv salvo com sucesso!


,nb,lr,svm
alpha,0.1,NaN,NaN
fit_prior,False,NaN,NaN
score,0.968148,0.995741,0.997778
C,NaN,1.0,0.1
solver,NaN,liblinear,NaN
penalty,NaN,l2,NaN
max_iter,NaN,200,5000
loss,NaN,NaN,squared_hinge
tol,NaN,NaN,0.001


In [9]:
# Treina modelo com melhores parâmetros
metrics = {}

for item in best_greedy:
    # Remove key score
    try:
        best_greedy[item].pop('score')
    except:
        pass
    metrics[item] = a3.train(model_name=item, params=best_greedy[item], series=series, base=BASE)

a3.save_data_frame(pd.DataFrame(metrics), BASE + '_metrics.csv')

# Exibe valores no notebook
pd.DataFrame(metrics)

Arquivo out/Dmoz-Sports_metrics.csv salvo com sucesso!


,nb,lr,svm
f1_score_macro,0.830608,0.898276,0.906953
f1_score_micro,0.834074,0.898889,0.908148
accuracy,0.834074,0.898889,0.908148
